#Linguistic Tailoring Intervention Study - Bayesian Analysis


## Notebook Overview

This notebook contains the full Bayesian analysis for the *Linguistic Tailoring intervention study*, including data preparation, model specification, model fitting, diagnostics, and model comparison using LOO cross-validation. The aim is to evaluate whether the full models provide better predictive performance than the null models.

IV: Planned-contrast: Intervention contrast (no intervention vs. intervention message). Tailoring contrast (non-tailored message vs. tailored message) * time (per day)
DV: Plant-based lunch choice (yes / no)
Random Intercepts: Per person
<br><br>
### **Study Model Structure**

**Independent Variables (planned contrasts):**

*   *Intervention contrast:* no intervention vs. intervention message
*   *Tailoring contrast:* non-tailored message vs. tailored message
*   *Time:* day-level variation
*   *Interaction:* Tailoring × Time

**Dependent Variable:** Plant-based lunch choice (yes/no)

**Random Effects:** Random intercepts for participants
<br><br>
### **Structure of Notebook**

1. **Hypothesis 1:** Tailoring × Intervention Contrast
     *   Between-subjects analyses
      *   Within-subjects analyses

2. **Hypothesis 2:** Message reponses
    *   Message Liking
    *   Message Shareability
    *   Message Relevance
    *   Perceived Personalisation
    *   Perceived Lingusitic Similarity

3. **Hypothesis 3:** Source reponses.
    *   Source Liking
    *   Source Trust
    *   Perceived Source Similarity

4. **Exploratory Analyses for Intervention Week 2**
<br>

In [ ]:
# General config & paths

from pathlib import Path

# Project root = folder where this notebook lives
PROJECT_ROOT = Path().resolve()

# Data folders (relative paths)
DATA_DIR = PROJECT_ROOT / "data"
PROCESSED_DIR = DATA_DIR / "processed"

In [2]:
import pandas as pd

In [3]:
import pandas as pd
!pip install bambi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 3.1 MB/s eta 0:00:00


In [4]:
# Now we can import bambi
import bambi as bmb

In [7]:
#Loading the first dataframe for the daily message survyes
df = pd.read_csv(PROCESSED_DIR / "daily_surveys_filtered2_python3.csv")

In [8]:
df[["Intervention_contrast", "Tailored_contrast", "condition", "week"]] \
    .drop_duplicates() \
    .sort_values(by="condition") \
    .reset_index(drop=True)

,Intervention_contrast,Tailored_contrast,condition,week
0,-0.66666,0.0,1.0,2
1,-0.66666,0.0,1.0,1
2,0.33333,-0.5,2.0,1
3,0.33333,-0.5,2.0,2
4,0.33333,0.5,3.0,1
5,0.33333,0.5,3.0,2
6,0.33333,-0.5,4.0,2
7,0.33333,0.5,4.0,1
8,0.33333,-0.5,5.0,1
9,0.33333,0.5,5.0,2


In [9]:
#Read second dataframe of the weekend surveys

df2 = pd.read_csv(PROCESSED_DIR /"weekend_surveys_python3.csv")

In [ ]:
#Defining Helper Functions for Bayesian Analysis

# ---------- helpers to list terms & extract draws ----------
def _all_scalar_params(idata):
    """Return scalar posterior parameters (chain, draw) likely to be fixed effects."""
    scalars = []
    for name, da in idata.posterior.data_vars.items():
        if set(("chain","draw")).issubset(da.dims) and da.ndim == 2:
            scalars.append(name)
    return scalars

def _beta_terms_if_present(idata):
    for packed in ["beta", "common", "coef"]:
        if packed in idata.posterior:
            da = idata.posterior[packed]
            for coord in ["term", "covariate", "feature", "predictor", "beta_dim"]:
                if coord in da.coords:
                    return list(map(str, da[coord].values))
    return None

def list_fixed_terms(idata, include_intercept=False):
    # prefer packed-coord names if present (most robust)
    terms = _beta_terms_if_present(idata)
    if terms is None:
        # fall back to scalar vars (exclude typical non-fixed params)
        terms = [t for t in _all_scalar_params(idata)
                 if not (t.startswith("sd_") or t.startswith("L_") or t.startswith("chol") or t.startswith("sigma"))]
    if not include_intercept:
        terms = [t for t in terms if t != "Intercept"]
    # keep unique order
    seen, out = set(), []
    for t in terms:
        if t not in seen:
            seen.add(t); out.append(t)
    return out

def draws_for(idata, term):
    # direct variable
    if term in idata.posterior.data_vars:
        return idata.posterior[term].values.reshape(-1)
    # packed arrays
    for packed in ["beta", "common", "coef"]:
        if packed in idata.posterior:
            da = idata.posterior[packed]
            for coord in ["term", "covariate", "feature", "predictor", "beta_dim"]:
                if coord in da.coords and term in list(map(str, da[coord].values)):
                    return da.sel({coord: term}).values.reshape(-1)
    raise KeyError(f"Could not find posterior draws for term: {term}")


In [ ]:
# --- robust helpers that also handle categorical (vector) coefficients

LEVEL_COORD_CANDIDATES = [
    "term","covariate","feature","predictor","beta_dim",
    "level","levels","category","categories"
]

def _is_param_we_want(varname):
    if varname in {"lp__"}:
        return False
    if varname.startswith(("sd_", "L_", "chol")):
        return False
    if varname.endswith("_sigma"):
        return False
    if "|" in varname:  # random-effect parts often include '|'
        return False
    if varname.lower() in {"sigma","tau","alpha"}:
        return False
    return True

def list_fixed_terms(idata, include_intercept=False):
    # 1) Try packed arrays first
    for packed in ["beta","common","coef"]:
        if packed in idata.posterior:
            da = idata.posterior[packed]
            for coord in LEVEL_COORD_CANDIDATES:
                if coord in da.coords:
                    terms = list(map(str, da[coord].values))
                    if not include_intercept:
                        terms = [t for t in terms if t != "Intercept"]
                    seen, out = set(), []
                    for t in terms:
                        if t not in seen:
                            seen.add(t); out.append(t)
                    return out
    # 2) Fall back: walk all variables, expand by level coords if present
    terms = []
    for name, da in idata.posterior.data_vars.items():
        if not _is_param_we_want(name):
            continue
        if set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if not other_dims:
                if include_intercept or name != "Intercept":
                    terms.append(name)
            elif len(other_dims) == 1:
                coord = other_dims[0]
                for level in map(str, da.coords[coord].values):
                    tname = f"{name}[{level}]"
                    if include_intercept or tname != "Intercept":
                        terms.append(tname)
            else:
                continue
    seen, out = set(), []
    for t in terms:
        if t not in seen:
            seen.add(t); out.append(t)
    return out

def draws_for(idata, term):
    # Packed arrays path
    for packed in ["beta","common","coef"]:
        if packed in idata.posterior:
            da = idata.posterior[packed]
            for coord in LEVEL_COORD_CANDIDATES:
                if coord in da.coords:
                    term_vals = list(map(str, da[coord].values))
                    if term in term_vals:
                        return da.sel({coord: term}).values.reshape(-1)
    # Unpacked path with level, e.g. "Dietary_identity[Flexitarian]"
    if "[" in term and term.endswith("]"):
        base, level = term.split("[", 1)
        base = base
        level = level[:-1]
        if base in idata.posterior:
            da = idata.posterior[base]
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                coord_vals_str = list(map(str, da.coords[coord].values))
                if level in coord_vals_str:
                    return da.sel({coord: level}).values.reshape(-1)
    # Simple scalar var
    if term in idata.posterior.data_vars:
        return idata.posterior[term].values.reshape(-1)
    raise KeyError(f"Could not find posterior draws for term: {term}")


# 1. Hypothesis 1: Intervention X Tailoring Contrast (Between Subjects)

First we estimate our main test, testing the effect of linguisitc tailoring on plant-based lunch choices of adolescents. First we will assess the between subjects effects from all research arms.

In [ ]:
df['condition'].value_counts()

,count
condition,
3.0,1540
1.0,1530
2.0,1520
5.0,780
4.0,760


In [13]:
#Defining Model 1 (Hypothesis 1a and b)

import pandas as pd, numpy as np, bambi as bmb, arviz as az, pymc as pm

df_model = df

df_model = df_model[df_model['condition'].isin([1, 2, 3, 4, 5])] #The first analysis contains the data from all 5 arms


# --- data ---
df_model = df_model[[
    "Lunch_plantbased2","Intervention_contrast","Tailored_contrast", "Day3",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Lunch_plantbased2"] = df_model["Lunch_plantbased2"].astype(int)
df_model["Name"] = df_model["Name"].astype("category")

# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


p0 = float(df_model["Lunch_plantbased2"].mean())
logit_p0 = float(np.log(p0/(1-p0))) if 0 < p0 < 1 else 0.0

priors = {
    "Intercept": bmb.Prior("Normal", mu=logit_p0, sigma=1.0),  # or wider if you want less constraint
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),         # applies to all fixed effects
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}

# Full: includes interaction and both mains
full_model = bmb.Model(
    "Lunch_plantbased2 ~ Intervention_contrast*Day3 + Tailored_contrast*Day3 +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="bernoulli", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Lunch_plantbased2 ~ Intervention_contrast + Tailored_contrast*Day3 +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="bernoulli", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

Output()

/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank     elpd_loo       p_loo  elpd_diff        weight         se  \
full     0 -3146.841464  406.685843   0.000000  1.000000e+00  31.786926   
null     1 -3153.360301  407.939738   6.518838  1.598721e-14  31.552818   

           dse  warning scale  
full  0.000000     True   log  
null  3.103419     True   log  


The LOO comparison slightly favored the model including the interaction term. The difference in expected log predictive density was strong, Δelpd_loo = 7.99 (dse = 3.13), corresponding to ΔLOOIC ≈ 6.83 (lower is better). Model complexity was similar (p_loo: 408.39 vs. 406.46), and stacking weights moderately preferred the full model (0.86 vs. 0.14). Overall, this suggests weak-to-moderate evidence that including the interaction improves out-of-sample predictive performance.

In [14]:
#Calculating the Bayes Factor

from scipy.stats import norm, gaussian_kde
import arviz as az

# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Intervention_contrast:Day3"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 35.39
BF_01 (Null over Full): 0.03
This mean that the Full model is 35.39 more likely than the null model.


In [17]:
# ================================
# Posterior summaries (fixed & random)
# ================================
import numpy as np
import pandas as pd
import arviz as az
from scipy.stats import norm, gaussian_kde

# ---------- config ----------
HDI = 0.95
BF_PRIOR_SD = 0.20   # weakly-informative prior on log-odds for fixed effects

# ---------- helpers for fixed effects (handles packed arrays & categorical levels) ----------
LEVEL_COORD_CANDIDATES = [
    "term","covariate","feature","predictor","beta_dim",
    "level","levels","category","categories"
]

def _is_param_we_want(varname):
    if varname in {"lp__"}:
        return False
    if varname.startswith(("sd_", "L_", "chol")):
        return False
    if varname.endswith("_sigma"):
        return False
    if "|" in varname:      # exclude random-effect arrays here; we'll handle them separately
        return False
    if varname.lower() in {"sigma","tau","alpha"}:
        return False
    return True

def list_fixed_terms(idata, include_intercept=False):
    # Try packed arrays first (bambi typically stores fixed effects here)
    for packed in ["beta","common","coef"]:
        if packed in idata.posterior:
            da = idata.posterior[packed]
            for coord in LEVEL_COORD_CANDIDATES:
                if coord in da.coords:
                    terms = list(map(str, da[coord].values))
                    if not include_intercept:
                        terms = [t for t in terms if t != "Intercept"]
                    # unique, keep order
                    seen, out = set(), []
                    for t in terms:
                        if t not in seen:
                            seen.add(t); out.append(t)
                    return out
    # Fallback: walk individual variables
    terms = []
    for name, da in idata.posterior.data_vars.items():
        if not _is_param_we_want(name):
            continue
        if set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if not other_dims:
                if include_intercept or name != "Intercept":
                    terms.append(name)
            elif len(other_dims) == 1:
                coord = other_dims[0]
                for level in map(str, da.coords[coord].values):
                    tname = f"{name}[{level}]"
                    if include_intercept or tname != "Intercept":
                        terms.append(tname)
            # ignore 2D+ params here
    seen, out = set(), []
    for t in terms:
        if t not in seen:
            seen.add(t); out.append(t)
    return out

def draws_for(idata, term):
    # Packed array path
    for packed in ["beta","common","coef"]:
        if packed in idata.posterior:
            da = idata.posterior[packed]
            for coord in LEVEL_COORD_CANDIDATES:
                if coord in da.coords:
                    vals = list(map(str, da[coord].values))
                    if term in vals:
                        return da.sel({coord: term}).values.reshape(-1)
    # Unpacked categorical path like "Factor[Level]"
    if "[" in term and term.endswith("]"):
        base, level = term.split("[", 1)
        level = level[:-1]
        if base in idata.posterior:
            da = idata.posterior[base]
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                coord_vals_str = list(map(str, da.coords[coord].values))
                if level in coord_vals_str:
                    return da.sel({coord: level}).values.reshape(-1)
    # Simple scalar var
    if term in idata.posterior.data_vars:
        return idata.posterior[term].values.reshape(-1)
    raise KeyError(f"Could not find posterior draws for term: {term}")

# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-1502604744.py:127: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in         BF10   logBF10
                                               Baseline_carnivore 1.20 [1.04, 1.45] 0.989    0.13         4.81  1.570354
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 1.06 [0.75, 1.58] 0.640    0.37         1.03  0.032269
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.22 [0.96, 1.64] 0.939    0.19         2.24  0.806537
                                               Children_age_years 1.04 [0.95, 1.13] 0.795    0.92         0.30 -1.207119
                           Country_of_residence[Northern Ireland] 1.00 [0.68, 1.42] 0.511    0.40         0.94 -0.065239
                        Country_of_residence[Republic of Ireland] 0.82 [0.60, 1.13] 0.866    0.24         1.57  0.452316
                                   Country_of_residence[Scotland] 0.91 [0.66, 1.24] 0.721    0.38         0.96 -0.043

In [19]:
# Total observations used in the model
N_obs = df_model.shape[0]

print(N_obs)


# Number of participants (random-intercept clusters)
N_participants = df_model["Name"].nunique()
print(N_participants)

5503
603


# 2. Hypothesis 1: Intervention X Tailoring Contrast (Within-Subjects)

Next we stimate the Within-Subjects Effects of Linguistic Tailoring on Plant-based lunch choice. For this we use data from the crossover groups 4a and 4b.

In [ ]:
df_model = df

df_model["condition"].value_counts()

,count
condition,
3.0,1540
1.0,1530
2.0,1520
5.0,780
4.0,760


In [ ]:
import pandas as pd, numpy as np, bambi as bmb, arviz as az, pymc as pm

df_model = df

df_model = df_model[df_model['condition'].isin([4, 5])].copy() #For this model we only use the crossover groups 4 and 5

# --- data ---
df_model = df_model[[
    "Lunch_plantbased2","Intervention_contrast","Tailored_contrast", "week", "Day2",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Lunch_plantbased2"] = df_model["Lunch_plantbased2"].astype(int)
df_model["Name"] = df_model["Name"].astype("category")

# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


p0 = float(df_model["Lunch_plantbased2"].mean())
logit_p0 = float(np.log(p0/(1-p0))) if 0 < p0 < 1 else 0.0

priors = {
    "Intercept": bmb.Prior("Normal", mu=logit_p0, sigma=1.0),  # or wider if you want less constraint
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),         # applies to all fixed effects
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}

# Full: includes interaction and both mains
full_model = bmb.Model(
    "Lunch_plantbased2 ~ Tailored_contrast*Day2*week +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="bernoulli", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Lunch_plantbased2 ~ Tailored_contrast + Day2 + week + "
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="bernoulli", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

Output()

/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank    elpd_loo       p_loo  elpd_diff  weight         se       dse  \
null     0 -823.505910  102.915705   0.000000     1.0  15.142406  0.000000   
full     1 -824.341955  105.001448   0.836046     0.0  15.242965  0.801259   

      warning scale  
null    False   log  
full     True   log  


The LOO comparison shows that the null model has the highest expected predictive accuracy (ELPD = –823.51) and receives essentially all the model weight (1.0), while the full model performs slightly worse (ELPD difference = –0.84). Because the ELPD difference is very small relative to its uncertainty (dSE ≈ 0.80), there is no meaningful evidence that the full model predicts better than the null model.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:Day2:week"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.59
BF_01 (Null over Full): 1.70
This mean that the Full model is 0.59 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in BF10   logBF10
                                               Baseline_carnivore 1.26 [1.00, 1.62] 0.967    0.15 3.28  1.187044
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 1.03 [0.66, 1.50] 0.567    0.36 1.05  0.046113
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 0.95 [0.69, 1.35] 0.609    0.38 0.97 -0.034332
                                               Children_age_years 0.96 [0.83, 1.14] 0.683    0.71 0.47 -0.751766
                           Country_of_residence[Northern Ireland] 0.92 [0.62, 1.36] 0.658    0.34 1.13  0.118885
                        Country_of_residence[Republic of Ireland] 0.94 [0.63, 1.37] 0.633    0.39 1.01  0.014427
                                   Country_of_residence[Scotland] 0.98 [0.68, 1.38] 0.543    0.40 0.96 -0.042405
                                      Country_of_residence[W

In [ ]:
# Total observations used in the model
N_obs = df_model.shape[0]

print(N_obs)


# Number of participants (random-intercept clusters)
N_participants = df_model["Name"].nunique()
print(N_participants)

1394
152


# 3. Message Responses (H2)

Next we assess the five message response variables message liking, shareability, relevance, perceived personalisation, perceived linguistic similarity.

## a) Message Liking

In [ ]:
df_model["Dietary_identity"].value_counts()

,count
Dietary_identity,
Omnivore,4545
Pescatarian,849
Flexitarian,109


In [ ]:
import pandas as pd, numpy as np, bambi as bmb, arviz as az, pymc as pm

df_model = df

# --- data ---
df_model = df_model[[
    "Lunch_plantbased2","Message_liking", "Intervention_contrast","Tailored_contrast", "Day3",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Lunch_plantbased2"] = df_model["Lunch_plantbased2"].astype(int)
df_model["Name"] = df_model["Name"].astype("category")

# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


mu0 = float(df_model["Message_liking"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}

# Full: includes interaction and both mains
full_model = bmb.Model(
    "Message_liking ~ Tailored_contrast*Day3 +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Message_liking ~ Tailored_contrast + Day3 + "
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))



Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO pos

      rank     elpd_loo       p_loo  elpd_diff    weight         se       dse  \
full     0 -2180.248227  397.658369   0.000000  0.813771  55.108630  0.000000   
null     1 -2181.843368  398.022047   1.595142  0.186229  55.198656  2.254985   

      warning scale  
full     True   log  
null     True   log  


The full model shows slightly better predictive accuracy than the null model (ELPD difference = 1.60) and receives most of the model weight (~0.81), but the uncertainty around this difference is large (dSE ≈ 2.25). Because the ELPD improvement is small relative to its uncertainty, the evidence favoring the full model over the null remains weak.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:Day3"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.10
BF_01 (Null over Full): 10.09
This mean that the Full model is 0.10 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in            BF10   logBF10
                                               Baseline_carnivore 1.05 [0.99, 1.12] 0.941    0.91            0.48 -0.743234
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.84 [0.68, 1.03] 0.936    0.24            1.74  0.555152
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.14 [1.03, 1.28] 0.998    0.25            6.26  1.834474
                                               Children_age_years 1.00 [0.98, 1.03] 0.610    1.00            0.08 -2.551736
                           Country_of_residence[Northern Ireland] 1.12 [0.86, 1.41] 0.800    0.39            1.05  0.046276
                        Country_of_residence[Republic of Ireland] 0.68 [0.58, 0.79] 1.000    0.00               ∞       inf
                                   Country_of_residence[Scotland] 0.96 [0.81, 1.10] 0.704    0.6

## b) Message Shareability

In [ ]:
df_model = df
# --- data ---
df_model = df_model[[
    "Lunch_plantbased2", "Message_shareability", "Message_liking", "Tailored_contrast", "Intervention_contrast","Day3",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Lunch_plantbased2"] = df_model["Lunch_plantbased2"].astype(int)
df_model["Name"] = df_model["Name"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})

# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})


mu0 = float(df_model["Message_shareability"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}


# Full: includes interaction and both mains
full_model = bmb.Model(
    "Message_shareability ~ Tailored_contrast* Day3  +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Message_shareability ~ Tailored_contrast +  Day3 +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO pos

      rank     elpd_loo       p_loo  elpd_diff  weight         se       dse  \
null     0 -2099.076701  401.533402   0.000000     1.0  48.141761  0.000000   
full     1 -2100.996740  404.362558   1.920039     0.0  48.240489  1.696715   

      warning scale  
null     True   log  
full     True   log  


The null model has the higher predictive accuracy (ELPD = –2099.08) and receives all the model weight, while the full model performs worse (ELPD difference = –1.92). Because this difference is small relative to its uncertainty (dSE ≈ 1.70), there is no meaningful evidence that the full model predicts better than the null model.

In [ ]:
from scipy.stats import norm, gaussian_kde
import arviz as az

# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:Day3"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.03
BF_01 (Null over Full): 34.49
This mean that the Full model is 0.03 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in         BF10   logBF10
                                               Baseline_carnivore 1.07 [0.99, 1.14] 0.961    0.80         0.92 -0.079522
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.89 [0.70, 1.11] 0.836    0.40         0.89 -0.114963
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.12 [1.00, 1.25] 0.973    0.36         1.77  0.569927
                                               Children_age_years 1.03 [0.99, 1.06] 0.932    1.00         0.22 -1.499660
                           Country_of_residence[Northern Ireland] 1.10 [0.86, 1.44] 0.790    0.43         0.82 -0.198944
                        Country_of_residence[Republic of Ireland] 0.58 [0.48, 0.69] 1.000    0.00            ∞       inf
                                   Country_of_residence[Scotland] 0.82 [0.69, 1.00] 0.976    0.14         3.29  1.190

## c) Message relevance

In [ ]:

df_model = df
# --- data ---
df_model = df_model[[
    "Lunch_plantbased2", "Message_relevance", "Message_shareability", "Message_liking", "Tailored_contrast", "Intervention_contrast","Day3",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Lunch_plantbased2"] = df_model["Lunch_plantbased2"].astype(int)
df_model["Name"] = df_model["Name"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})

mu0 = float(df_model["Message_relevance"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}


# Full: includes interaction and both mains
full_model = bmb.Model(
    "Message_relevance ~ Tailored_contrast* Day3  +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Message_relevance ~ Tailored_contrast + Day3 +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO pos

      rank     elpd_loo       p_loo  elpd_diff  weight         se       dse  \
null     0 -3506.046309  418.740423   0.000000     1.0  59.897751  0.000000   
full     1 -3510.562132  423.905348   4.515823     0.0  60.192036  1.924308   

      warning scale  
null     True   log  
full     True   log  


The null model shows higher out-of-sample predictive performance (ELPD = –3506.05) and receives all the model weight, while the full model performs worse by about 4.52 ELPD units. Although the difference is modest relative to its uncertainty (dSE ≈ 1.92), there is still no evidence that the full model improves prediction over the null model.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:Day3"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.04
BF_01 (Null over Full): 25.73
This mean that the Full model is 0.04 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in        BF10   logBF10
                                               Baseline_carnivore 1.14 [1.03, 1.27] 0.991    0.25        3.87  1.354529
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.85 [0.63, 1.16] 0.846    0.28        1.27  0.241575
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.07 [0.90, 1.27] 0.772    0.59        0.60 -0.507503
                                               Children_age_years 1.03 [0.98, 1.08] 0.873    0.99        0.24 -1.426882
                           Country_of_residence[Northern Ireland] 1.07 [0.76, 1.42] 0.651    0.41        0.88 -0.124955
                        Country_of_residence[Republic of Ireland] 0.57 [0.45, 0.73] 1.000    0.00 41217863.00 17.534382
                                   Country_of_residence[Scotland] 0.87 [0.68, 1.12] 0.872    0.34        1.14  0.134965
    

## d) Perceived Personalization

In [ ]:

df_model2 = df2
# --- data ---
df_model = df_model2[[
    "Message_personalized", "Tailored_contrast", "Intervention_contrast","week",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Name"] = df_model["Name"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})

mu0 = float(df_model["Message_personalized"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}


# Full: includes interaction and both mains
full_model = bmb.Model(
    "Message_personalized ~ Tailored_contrast*week  +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Message_personalized ~ Tailored_contrast + week +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

Output()

/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank     elpd_loo       p_loo  elpd_diff    weight         se       dse  \
full     0 -1124.591020  319.749793   0.000000  0.848355  29.090103  0.000000   
null     1 -1127.477793  323.299880   2.886773  0.151645  29.338234  2.911769   

      warning scale  
full     True   log  
null     True   log  


/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


The full model has better out-of-sample predictive performance (ELPD = –1124.59) and receives most of the model weight (~0.85), outperforming the null model by about 2.89 ELPD units. However, because this difference is close to its uncertainty (dSE ≈ 2.91), the evidence favoring the full model is modest rather than decisive.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:week"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.53
BF_01 (Null over Full): 1.87
This mean that the Full model is 0.53 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in            BF10   logBF10
                                               Baseline_carnivore 1.28 [1.12, 1.48] 1.000    0.02          917.79  6.821965
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.94 [0.72, 1.41] 0.630    0.41            0.96 -0.038794
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.11 [0.88, 1.38] 0.812    0.41            0.96 -0.042058
                                               Children_age_years 1.02 [0.95, 1.09] 0.697    0.99            0.21 -1.569852
                           Country_of_residence[Northern Ireland] 1.06 [0.72, 1.45] 0.616    0.39            0.99 -0.012472
                        Country_of_residence[Republic of Ireland] 0.61 [0.46, 0.82] 0.998    0.00           64.49  4.166533
                                   Country_of_residence[Scotland] 0.86 [0.66, 1.19] 0.838    0.3

## e) Perceived Linguistic Similarity

In [ ]:

df_model2 = df2
# --- data ---
df_model = df_model2[[
    "Message_similarity", "Tailored_contrast", "Intervention_contrast","week",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Name"] = df_model["Name"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})


mu0 = float(df_model["Message_similarity"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}


# Full: includes interaction and both mains
full_model = bmb.Model(
    "Message_similarity ~ Tailored_contrast*week  +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Message_similarity ~ Tailored_contrast + week +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

Output()

/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank     elpd_loo       p_loo  elpd_diff    weight         se       dse  \
full     0 -1187.181948  324.020598   0.000000  0.781543  23.325712  0.000000   
null     1 -1189.720752  322.895008   2.538804  0.218457  23.113605  3.015987   

      warning scale  
full     True   log  
null     True   log  


/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


The full model shows slightly better out-of-sample predictive performance (ELPD = –1187.18) and receives most of the model weight (~0.78), outperforming the null model by about 2.54 ELPD units. However, because this difference is small relative to its uncertainty (dSE ≈ 3.02), the evidence favoring the full model is weak rather than conclusive.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:week"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.64
BF_01 (Null over Full): 1.57
This mean that the Full model is 0.64 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in             BF10   logBF10
                                               Baseline_carnivore 1.31 [1.14, 1.49] 0.999    0.01           570.87  6.347158
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.92 [0.67, 1.24] 0.696    0.39             0.96 -0.036214
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.15 [0.91, 1.47] 0.861    0.33             1.12  0.117265
                                               Children_age_years 0.97 [0.90, 1.04] 0.795    0.96             0.26 -1.360372
                           Country_of_residence[Northern Ireland] 0.98 [0.68, 1.41] 0.539    0.39             0.96 -0.037282
                        Country_of_residence[Republic of Ireland] 0.59 [0.44, 0.80] 1.000    0.00           702.72  6.554954
                                   Country_of_residence[Scotland] 0.83 [0.62, 1.11] 0.864

# 4. Source Responses (H3)

Lastly, we test the effect of linguistic tailoring on source-level responses a) source liking, b) source trust, and c) perceived source similarity.

## a) Source Liking

In [ ]:
import pandas as pd, numpy as np, bambi as bmb, arviz as az, pymc as pm

df_model2 = df2
# --- data ---
df_model = df_model2[[
    "Source_liking", "Tailored_contrast", "Intervention_contrast","week",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Name"] = df_model["Name"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})


mu0 = float(df_model["Source_liking"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}



# Full: includes interaction and both mains
full_model = bmb.Model(
    "Source_liking ~ Tailored_contrast*week  +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Source_liking ~ Tailored_contrast + week +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

Output()

/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank    elpd_loo       p_loo  elpd_diff  weight         se       dse  \
null     0 -393.373387  332.241989   0.000000     1.0  22.780044  0.000000   
full     1 -400.027483  337.084008   6.654096     0.0  23.240483  3.226082   

      warning scale  
null     True   log  
full     True   log  


/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


The null model clearly outperforms the full model (ELPD difference = –6.65) and receives all the model weight. Although the uncertainty is moderate (dSE ≈ 3.23), the difference is large enough to conclude that the full model does not improve predictive performance.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:week"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.22
BF_01 (Null over Full): 4.46
This mean that the Full model is 0.22 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in     BF10   logBF10
                                               Baseline_carnivore 1.15 [1.07, 1.24] 1.000    0.11 72063.74 11.185306
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.94 [0.74, 1.19] 0.704    0.51     0.71 -0.346103
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.10 [0.95, 1.24] 0.904    0.51     0.81 -0.215615
                                               Children_age_years 1.03 [0.99, 1.06] 0.918    1.00     0.23 -1.474535
                           Country_of_residence[Northern Ireland] 1.00 [0.74, 1.28] 0.505    0.51     0.70 -0.352321
                        Country_of_residence[Republic of Ireland] 0.69 [0.58, 0.85] 1.000    0.00   126.18  4.837720
                                   Country_of_residence[Scotland] 0.83 [0.68, 0.98] 0.982    0.15     4.67  1.540739
                            

## b) Source Trust

In [ ]:

df_model2 = df2
# --- data ---
df_model = df_model2[[
    "Source_trust", "Tailored_contrast", "Intervention_contrast","week",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Name"] = df_model["Name"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})


mu0 = float(df_model["Source_trust"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}

# Full: includes interaction and both mains
full_model = bmb.Model(
    "Source_trust ~ Tailored_contrast*week  +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Source_trust ~ Tailored_contrast + week +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details


Output()

/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank    elpd_loo       p_loo  elpd_diff    weight         se       dse  \
null     0 -369.578251  340.953847   0.000000  0.886569  28.091412  0.000000   
full     1 -376.684147  345.742430   7.105895  0.113431  28.346068  4.365678   

      warning scale  
null     True   log  
full     True   log  


/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


The null model shows better out-of-sample predictive performance (ELPD = –369.58) and receives most of the model weight (~0.89), while the full model performs worse by about 7.11 ELPD units. Given that this difference is larger than its uncertainty (dSE ≈ 4.37), there is no evidence that the full model improves prediction and some evidence that it performs meaningfully worse.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:week"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.22
BF_01 (Null over Full): 4.47
This mean that the Full model is 0.22 more likely than the null model.


In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in   BF10   logBF10
                                               Baseline_carnivore 1.14 [1.06, 1.23] 1.000    0.19  23.04  3.137446
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.95 [0.74, 1.21] 0.696    0.50   0.68 -0.381138
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.09 [0.95, 1.25] 0.869    0.56   0.72 -0.323063
                                               Children_age_years 1.02 [0.99, 1.06] 0.846    1.00   0.16 -1.847751
                           Country_of_residence[Northern Ireland] 1.04 [0.77, 1.35] 0.634    0.50   0.70 -0.350044
                        Country_of_residence[Republic of Ireland] 0.68 [0.55, 0.83] 1.000    0.00 852.40  6.748058
                                   Country_of_residence[Scotland] 0.89 [0.73, 1.06] 0.900    0.39   0.98 -0.025176
                                      Countr

## c) Perceived Source Similarity

In [ ]:
df_model2 = df2
# --- data ---
df_model = df_model2[[
    "Perceived_similarity", "Tailored_contrast", "Intervention_contrast","week",
    "Dietary_identity","SES","Country_of_residence",
    "Gender_dummy","Imposter","Children_age_years",
    "Baseline_carnivore","Lunch_baseline","Name"
]].dropna().copy()

df_model["Name"] = df_model["Name"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].astype("category")
df_model["Country_of_residence"] = df_model["Country_of_residence"].cat.rename_categories({
    1: "England",
    2: "Northern Ireland",
    3: "Republic of Ireland",
    4: "Scotland",
    5: "Wales"
})


# Convert to categorical and relabel
df_model["Dietary_identity"] = df_model["Dietary_identity"].astype("category")
df_model["Dietary_identity"] = df_model["Dietary_identity"].cat.rename_categories({
    1: "Omnivore",
    2: "Flexitarian",
    3: "Pescatarian"
})


mu0 = float(df_model["Perceived_similarity"].mean())
priors = {
    "Intercept": bmb.Prior("Normal", mu=mu0, sigma=1.0),
    "common":    bmb.Prior("Normal", mu=0, sigma=0.20),
    "group_specific": bmb.Prior("Normal", mu=0, sigma=bmb.Prior("HalfStudentT", nu=3, sigma=1.5)),
}

# Full: includes interaction and both mains
full_model = bmb.Model(
    "Perceived_similarity ~ Tailored_contrast*week  +"
    "C(Dietary_identity, Treatment(reference='Omnivore')) + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

# Reduced (no-interaction): keep both main effects X and Day3
null_model = bmb.Model(
    "Perceived_similarity ~ Tailored_contrast + week +"
    "Dietary_identity + SES + Country_of_residence + "
    "Gender_dummy + Imposter + Children_age_years + "
    "Baseline_carnivore + Lunch_baseline + (1|Name)",
    data=df_model, family="gaussian", priors=priors
)

full_idata = full_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=11,
                            idata_kwargs={"log_likelihood": True})
null_idata = null_model.fit(draws=500, tune=2000, target_accept=0.9, random_seed=10,
                            idata_kwargs={"log_likelihood": True})

print(az.compare({"full": full_idata, "null": null_idata}))


Output()

Output()

ERROR:pymc.stats.convergence:The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


      rank    elpd_loo       p_loo  elpd_diff    weight         se       dse  \
null     0 -974.525511  351.539805   0.000000  0.601017  26.415572  0.000000   
full     1 -975.472470  351.869756   0.946959  0.398983  25.298804  4.027666   

      warning scale  
null     True   log  
full     True   log  


/usr/local/lib/python3.12/dist-packages/arviz/stats/stats.py:797: UserWarning: Estimated shape parameter of Pareto distribution is greater than 0.67 for one or more samples. You should consider using a more robust model, this is because importance sampling is less likely to work well if the marginal posterior and LOO posterior are very different. This is more likely to happen with a non-robust model and highly influential observations.
  warnings.warn(


The null and full models have very similar predictive performance, with the null model slightly ahead (ELPD difference = 0.95) and receiving about 60% of the model weight. However, because the difference is tiny relative to its uncertainty (dSE ≈ 4.03), there is no meaningful evidence favoring one model over the other.

In [ ]:
# Step 1: Extract posterior samples of the interaction term
samples = full_idata.posterior["Tailored_contrast:week"].values.flatten()

# Step 2: Estimate posterior density at 0
posterior_kde = gaussian_kde(samples)
posterior_at_0 = posterior_kde.evaluate(0)[0]

# Step 3: Define the prior: Normal(0, 0.20)
prior_at_0 = norm.pdf(0, loc=0, scale=0.20)

# Step 4: Calculate Bayes Factor
BF_10 = prior_at_0 / posterior_at_0
BF_01 = 1 / BF_10

print(f"BF_10 (Full over Null): {BF_10:.2f}")
print(f"BF_01 (Null over Full): {BF_01:.2f}")

print(f"This mean that the Full model is {BF_10:.2f} more likely than the null model.")

BF_10 (Full over Null): 0.77
BF_01 (Null over Full): 1.30
This mean that the Full model is 0.77 more likely than the null model.


--

In [ ]:
# ---------- fixed effects summary (OR, CrI, pd, ROPE, BF via Savage–Dickey) ----------
terms = list_fixed_terms(full_idata, include_intercept=False)

rows = []
for term in terms:
    arr = draws_for(full_idata, term)                       # log-odds draws
    med = float(np.median(arr))
    lo, hi = map(float, az.hdi(arr, HDI))
    pd_val = float(max((arr > 0).mean(), (arr < 0).mean())) # posterior probability of direction
    rows.append({
        "term": term,
        "OR_median": float(np.exp(med)),
        "OR_low95": float(np.exp(lo)),
        "OR_high95": float(np.exp(hi)),
        "pd": pd_val
    })
fixed_table = pd.DataFrame(rows)

# ROPE on OR scale ±10% around 1.0 -> log-odds bounds:
rope_low, rope_high = -np.log(1.10), np.log(1.10)
fixed_table["ROPE_in"] = fixed_table["term"].apply(
    lambda t: float(((draws_for(full_idata, t) >= rope_low) &
                     (draws_for(full_idata, t) <= rope_high)).mean())
)

# Bayes factor per term (Savage–Dickey at 0 on log-odds)
def bf10_savage_dickey(term, prior_sd=BF_PRIOR_SD):
    arr = draws_for(full_idata, term)
    kde = gaussian_kde(arr)            # posterior density at 0
    post0 = float(kde.evaluate(0.0))
    prior0 = float(norm.pdf(0.0, loc=0.0, scale=prior_sd))
    if post0 <= 1e-12:
        return np.inf, np.inf
    BF10 = prior0 / post0
    return BF10, np.log(BF10)

BF = [bf10_savage_dickey(t, prior_sd=BF_PRIOR_SD) for t in fixed_table["term"]]
fixed_table["BF10"] = [v[0] for v in BF]
fixed_table["logBF10"] = [v[1] for v in BF]

# Pretty print (optional)
fixed_out = fixed_table.copy()
fixed_out["OR (95% CrI)"] = fixed_out.apply(
    lambda r: f"{r['OR_median']:.2f} [{r['OR_low95']:.2f}, {r['OR_high95']:.2f}]",
    axis=1
)
fixed_out["pd"] = fixed_out["pd"].map(lambda x: f"{x:.3f}")
fixed_out["ROPE_in"] = fixed_out["ROPE_in"].map(lambda x: f"{x:.2f}")
fixed_out["BF10"] = fixed_out["BF10"].map(lambda x: "∞" if np.isinf(x) else f"{x:.2f}")
fixed_out = fixed_out[["term","OR (95% CrI)","pd","ROPE_in","BF10","logBF10"]].sort_values("term")

print("\n=== FIXED EFFECTS (full model) ===")
print(fixed_out.to_string(index=False))

# ================================
# RANDOM INTERCEPTS for Name
# ================================
def _find_name_raneff_var(idata, group="Name"):
    """
    Return (varname, coord_name) for the random intercept array for given group.
    """
    for varname, da in idata.posterior.data_vars.items():
        if f"|{group}" in varname and set(("chain","draw")).issubset(da.dims):
            other_dims = [d for d in da.dims if d not in ("chain","draw")]
            if len(other_dims) == 1:
                coord = other_dims[0]
                return varname, coord
    raise KeyError(f"Could not find a random-effect array for group '{group}'")

def summarize_name_random_intercepts(idata, group="Name", hdi_prob=0.95):
    # Find group-level SD parameter
    sd_name = None
    for v in idata.posterior.data_vars:
        if v.endswith(f"|{group}_sigma") or v.endswith(f"|{group}__sigma") or v == f"1|{group}_sigma":
            sd_name = v
            break

    sd_summary = None
    if sd_name is not None:
        sd_draws = idata.posterior[sd_name].values.reshape(-1)
        s_lo, s_hi = map(float, az.hdi(sd_draws, hdi_prob))
        sd_summary = {
            "sd_param": sd_name,
            "sd_median": float(np.median(sd_draws)),
            f"sd_low{int(hdi_prob*100)}": s_lo,
            f"sd_high{int(hdi_prob*100)}": s_hi,
        }

    # Per-Name random intercept draws
    varname, coord = _find_name_raneff_var(idata, group=group)
    da = idata.posterior[varname]  # dims: chain, draw, <coord>
    names = list(map(str, da.coords[coord].values))

    rows = []
    for lvl in names:
        draws = da.sel({coord: lvl}).values.reshape(-1)  # log-odds shift (deviation from overall intercept)
        med = float(np.median(draws))
        lo, hi = map(float, az.hdi(draws, hdi_prob))
        rows.append({
            group: lvl,
            "logit_median": med,
            f"logit_low{int(hdi_prob*100)}": lo,
            f"logit_high{int(hdi_prob*100)}": hi,
            "OR_median": float(np.exp(med)),
            f"OR_low{int(hdi_prob*100)}": float(np.exp(lo)),
            f"OR_high{int(hdi_prob*100)}": float(np.exp(hi)),
        })

    df_re = pd.DataFrame(rows).sort_values("logit_median")
    return sd_summary, df_re

# Run random intercept summary
try:
    sd_summary, name_re = summarize_name_random_intercepts(full_idata, group="Name", hdi_prob=HDI)
    print("\n=== RANDOM INTERCEPTS: Name (group SD) ===")
    print(sd_summary)
    print("\nExamples (lowest 5 Name effects):")
    print(name_re.head(5).to_string(index=False))
    print("\nExamples (highest 5 Name effects):")
    print(name_re.tail(5).to_string(index=False))

    # Optional: save full table
    # name_re.to_csv("random_intercepts_Name.csv", index=False)

except KeyError as e:
    print("\n[Info] Could not locate the random intercept array for 'Name'.")
    print("       Check your model’s group specification or posterior variable names.")
    print(f"       Details: {e}")


/tmp/ipython-input-3902273959.py:30: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  post0 = float(kde.evaluate(0.0))



=== FIXED EFFECTS (full model) ===
                                                             term      OR (95% CrI)    pd ROPE_in     BF10   logBF10
                                               Baseline_carnivore 1.23 [1.07, 1.39] 1.000    0.06    20.54  3.022586
C(Dietary_identity, Treatment(reference='Omnivore'))[Flexitarian] 0.97 [0.73, 1.40] 0.554    0.41     0.90 -0.102111
C(Dietary_identity, Treatment(reference='Omnivore'))[Pescatarian] 1.12 [0.91, 1.42] 0.842    0.39     0.97 -0.033122
                                               Children_age_years 0.98 [0.91, 1.05] 0.729    0.98     0.24 -1.435251
                           Country_of_residence[Northern Ireland] 0.97 [0.69, 1.40] 0.568    0.40     0.92 -0.078711
                        Country_of_residence[Republic of Ireland] 0.65 [0.49, 0.87] 1.000    0.01    49.28  3.897530
                                   Country_of_residence[Scotland] 0.84 [0.64, 1.16] 0.871    0.26     1.54  0.430417
                            

In [ ]:
# Total observations used in the model
N_obs = df_model.shape[0]

print(N_obs)

905


# 5. Exploratory Analyses

Next, we ran some exploratory analyses exlcusively on week 2, given that it seemed like effects only appeared to build after the first week. For this we ran t-tests between conditions (tailroed vs. non tailored messaging).

## T-tests for Week 2

In [ ]:
!pip install pingouin
import pingouin as pg
import pandas as pd
import numpy as np


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.4/204.4 kB 7.0 MB/s eta 0:00:00


In [ ]:
# Filter for Week 2 (Day3 values 8–12)
week2_data = df[df["Day3"].between(8, 12)]

# Compute group means per participant
mean_week2 = week2_data.groupby(["Name", "Tailored_contrast"]).agg({
    "Message_liking": "mean",
    "Message_shareability": "mean",
    "Message_relevance": "mean",
    "Lunch_plantbased2": "mean"
}).reset_index()


In [ ]:
import numpy as np
import pingouin as pg
from scipy.stats import ttest_ind

def report_bayes_ttest(var, r=0.2):
    """
    Run one-sided (greater) Bayesian and frequentist t-tests
    comparing tailored > non-tailored, with prior scale r.
    Also reports group means.
    """
    group1 = mean_week2[mean_week2["Tailored_contrast"] == 0.5][var].dropna()
    group2 = mean_week2[mean_week2["Tailored_contrast"] == -0.5][var].dropna()

    mean1 = group1.mean()
    mean2 = group2.mean()

    # --- Frequentist Welch t-test
    t_stat, p_two = ttest_ind(group1, group2, equal_var=False)

    # Convert to one-sided p-value (greater: tailored > non-tailored)
    if t_stat > 0:
        p_val = p_two / 2
    else:
        p_val = 1 - (p_two / 2)

    # --- Bayesian t-test
    bf10 = pg.bayesfactor_ttest(
        t=float(t_stat), nx=len(group1), ny=len(group2),
        paired=False, alternative='greater', r=r
    )
    bf01 = 1 / bf10 if np.isfinite(bf10) and bf10 != 0 else np.inf

    # --- Print result
    print(f"Bayesian t-test for {var.replace('_', ' ').title()}:")
    print(f"  Mean (Tailored)     = {mean1:.2f}")
    print(f"  Mean (Non-tailored) = {mean2:.2f}")
    print(f"  t = {t_stat:.2f}, one-sided p = {p_val:.3f}")
    print(f"  BF10 = {bf10:.2f}, BF01 = {bf01:.2f} (r = {r})\n")


In [ ]:
# Example usage:
report_bayes_ttest("Message_liking", r=0.2)
report_bayes_ttest("Message_shareability", r=0.2)
report_bayes_ttest("Message_relevance", r=0.2)
report_bayes_ttest("Lunch_plantbased2", r=0.2)


Bayesian t-test for Message Liking:
  Mean (Tailored)     = 2.60
  Mean (Non-tailored) = 2.54
  t = 1.32, one-sided p = 0.095
  BF10 = 1.25, BF01 = 0.80 (r = 0.2)

Bayesian t-test for Message Shareability:
  Mean (Tailored)     = 2.50
  Mean (Non-tailored) = 2.40
  t = 1.84, one-sided p = 0.033
  BF10 = 2.43, BF01 = 0.41 (r = 0.2)

Bayesian t-test for Message Relevance:
  Mean (Tailored)     = 5.82
  Mean (Non-tailored) = 5.61
  t = 2.07, one-sided p = 0.019
  BF10 = 3.50, BF01 = 0.29 (r = 0.2)

Bayesian t-test for Lunch Plantbased2:
  Mean (Tailored)     = 0.58
  Mean (Non-tailored) = 0.56
  t = 0.61, one-sided p = 0.271
  BF10 = 0.74, BF01 = 1.36 (r = 0.2)



In [ ]:
import numpy as np
import pingouin as pg
from scipy.stats import ttest_ind

# Filter for week 2
week2_data = df2[df2["week"] == 2]

# Variables to test
variables = ["Source_liking", "Source_trust", "Perceived_similarity",
             "Message_personalized", "Message_similarity"]

# Prior scale for Bayesian t-test (Cauchy(0, r))
R_PRIOR = 0.2   # your "small effect" prior

for var in variables:
    tailored = week2_data[week2_data["Tailored_contrast"] == 0.5][var].dropna().to_numpy()
    non_tailored = week2_data[week2_data["Tailored_contrast"] == -0.5][var].dropna().to_numpy()

    n1, n2 = len(tailored), len(non_tailored)
    if n1 < 2 or n2 < 2:
        print(f"{var}: Not enough data (n_tailored={n1}, n_non_tailored={n2}).\n")
        continue

    # Frequentist Welch t-test (two-sided)
    t_stat, p_two = ttest_ind(tailored, non_tailored, equal_var=False)

    # Convert to one-sided p-value for alternative: tailored > non-tailored
    if t_stat > 0:
        p_val = p_two / 2
    else:
        p_val = 1 - (p_two / 2)

    # Welch-Satterthwaite df
    s1, s2 = np.var(tailored, ddof=1), np.var(non_tailored, ddof=1)
    df = (s1/n1 + s2/n2)**2 / ((s1**2)/((n1**2)*(n1 - 1)) + (s2**2)/((n2**2)*(n2 - 1)))

    # Bayesian t-test (Rouder JZS BF with custom prior r)
    BF10 = pg.bayesfactor_ttest(t=float(t_stat), nx=n1, ny=n2,
                                paired=False, alternative='greater', r=R_PRIOR)
    BF01 = 1 / BF10 if np.isfinite(BF10) and BF10 != 0 else np.inf

    # Descriptives
    mean_t, mean_nt = float(np.mean(tailored)), float(np.mean(non_tailored))

    print(f"{var}:")
    print(f"  Tailored mean = {mean_t:.2f}, Non-tailored mean = {mean_nt:.2f}")
    print(f"  Welch t({df:.1f}) = {t_stat:.2f}, one-sided p = {p_val:.3f}")
    print(f"  Bayes factor (r={R_PRIOR}): BF10 = {BF10:.2f}, BF01 = {BF01:.2f}\n")


Source_liking:
  Tailored mean = 3.48, Non-tailored mean = 3.36
  Welch t(451.2) = 2.05, one-sided p = 0.020
  Bayes factor (r=0.2): BF10 = 3.38, BF01 = 0.30

Source_trust:
  Tailored mean = 3.45, Non-tailored mean = 3.36
  Welch t(449.7) = 1.59, one-sided p = 0.056
  Bayes factor (r=0.2): BF10 = 1.71, BF01 = 0.58

Perceived_similarity:
  Tailored mean = 4.43, Non-tailored mean = 4.25
  Welch t(449.2) = 1.50, one-sided p = 0.068
  Bayes factor (r=0.2): BF10 = 1.53, BF01 = 0.65

Message_personalized:
  Tailored mean = 4.86, Non-tailored mean = 4.60
  Welch t(449.7) = 2.27, one-sided p = 0.012
  Bayes factor (r=0.2): BF10 = 4.93, BF01 = 0.20

Message_similarity:
  Tailored mean = 4.64, Non-tailored mean = 4.56
  Welch t(451.6) = 0.63, one-sided p = 0.264
  Bayes factor (r=0.2): BF10 = 0.75, BF01 = 1.34

